# Bottom-Up Cube (BUC) Algorithm

In [ ]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
from itertools import combinations
from memory_profiler import profile

## In-Memory Implementation

In [2]:
class BUC:
    def __init__(self,numDims,cardinality,minsup,outputRec,dataCount):
        self.numDims = numDims # The total number of  dimensions. 
        self.cardinality = cardinality # The cardinality of each  dimension. 
        self.minsup = minsup # The minimum number of  tuples in a partition for  it to  be  output. 
        self.outputRec = outputRec # The current output record. 
        self.dataCount = dataCount # Stores the size  of  each  partition. dataCount[i] is  a  list of  integers of  size cardinality[i].

    def aggregate(self,input):
        self.data = [list(x) for x in input]

    def Partition(self,input,d,C,dataCount):
        for i in range(C):
            dataCount[i] = 0 
        for i in range(len(input)):
            dataCount[input[i][d]] += 1
        for i in range(1,C):
            dataCount[i] += dataCount[i-1]

    def writeAncestors(self,dim):
        for i in range(dim):
            print(self.outputRec[i],end = " ")
        print()

    def run(self,input,dim):
        self.aggregate(input)
        input = self.data
        if len(input) == 1:
            self.writeAncestors(dim)
            return
        
        # write the output record
        for d in range(self.numDims):
            C = self.cardinality[d]
            self.Partition(input,d,C,self.dataCount[d],dim) 
            k = 0
            for i in range(C):
                c = self.dataCount[d][i]
                if c >= self.minsup:
                    self.outputRec[d] = input[k][d]
                    self.run(input[k:],dim+1)
                k += c
        self.outputRec[dim] = None
    
    def print_result(self):
        self.run(self.data,0)
    
    def get_result(self):
        return self.result
